In [1]:
## import dependencies

In [16]:
import gym
from gym import Env
from gym.spaces import Discrete,Box,Dict,Tuple,MultiBinary,MultiDiscrete
import random

# import helpers

import numpy as np
import os 

#import stable baselines

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env  import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

## building my own env
- built an agent to give the best shower

In [92]:
class ShowerEnv(Env):
    def __init__(self):
        self.action_space = Discrete(3)
        self.observation_space = Box(0,100,(1,))
        self.state = 38 + random.randint(-3,3)
        self.shower_lenth = 60 # amount of time we shower.
    
    def step(self,action):
        # the idea is if action =0 implies decrease 1 and act =1 no effect, act =2 implies we increase temp by 1 degere.
        self.state += action -1
        
        self.shower_lenth -= 1
        
        #reward
        if self.state>=37 and self.state<=39:
            reward =1
        else:
            reward = -1
        
        if self.shower_lenth<=0:
            done= True
        else:
            done = False
        info= {}
        
        return self.state, reward,done,info
        
    
        
    
    def render(self):
        pass
    
    def reset(self):
            self.shower_lenth = 60 # amount of time we shower.
            #reward =0
            self.state = np.array([38 + random.randint(-3,3)]).astype(float)
            return self.state

        

In [93]:
 Box(0,100,(1,)).sample()

array([96.08784], dtype=float32)

In [94]:
Discrete(3).sample()

0

In [95]:
env= ShowerEnv()

In [96]:
env.action_space.sample()

0

In [97]:
env.observation_space

Box(0.0, 100.0, (1,), float32)

In [98]:
episodes = 5
for episode in range(1,episodes+1):
    obs = env.reset()
    print(obs)
    done = False
    score =0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        obs,reward,done,info = env.step(action)
        score += reward
       
    print('episode:{} score :{}'.format(episode,score))
#env.close()

[39.]
episode:1 score :-26
[41.]
episode:2 score :8
[35.]
episode:3 score :-60
[38.]
episode:4 score :-28
[39.]
episode:5 score :-38


In [102]:
env= ShowerEnv()
#env = DummyVecEnv([lambda:env])
model = PPO('MlpPolicy',env,verbose=1)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [106]:
model.learn(total_timesteps=100000)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -37.1    |
| time/              |          |
|    fps             | 1823     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | -30.9       |
| time/                   |             |
|    fps                  | 1242        |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.011139424 |
|    clip_fraction        | 0.122       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.08       |
|    explained_variance   | -0.000122   |
|    learning_rate        | 0.

In [107]:
episodes = 5
for episode in range(1,episodes+1):
    obs = env.reset()
    #print(obs)
    done = False
    score =0 
    
    while not done:
        env.render()
        action,_ = model.predict(obs)
        obs,reward,done,info = env.step(action)
        score += reward
       
    print('episode:{} score :{}'.format(episode,score))
#env.close()

episode:1 score :54
episode:2 score :58
episode:3 score :50
episode:4 score :48
episode:5 score :58


In [108]:
model.save("models/shower_model")

In [109]:
evaluate_policy(model, env,n_eval_episodes=10)

C:\Users\nikta\anaconda3\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
C:\Users\nikta\anaconda3\lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(47.8, 35.93828042630866)